In [5]:
pip install selenium


SyntaxError: invalid syntax (<ipython-input-5-df8d2f0cc70c>, line 1)

In [6]:
pip install pymongo

     |████████████████████████████████| 527 kB 7.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [21]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [22]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [23]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver'

In [24]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

<ipython-input-24-c8b61cbdd697>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [25]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [26]:
# Getting HTML content for news listing page
soup = make_request(browser, '/archivo/politica/')
soup.append(make_request(browser, '/archivo/ciencia/'))
soup.append(make_request(browser, '/archivo/tecnologia/'))
soup.append(make_request(browser, '/archivo/salud/'))
soup.append(make_request(browser, '/archivo/educacion/'))

In [27]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [28]:
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)
print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion

3


In [29]:
# Finding and concatenating news cards
cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
len(cards)

4

In [30]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [32]:
news

[{'title': 'Jennifer Arias influyó en el voto de un representante en la Ley de Garantías',
  'relative_path': '/politica/jennifer-arias-influyo-en-el-voto-de-un-representante-a-la-ley-de-garantias/'},
 {'title': '“Bases de datos del DANE no son confiables”: registrador nacional ',
  'relative_path': '/economia/bases-de-datos-del-dane-no-son-confiables-registrador-nacional/'},
 {'title': 'Gobierno, a rendir cuentas sobre sobre atención a mujeres víctimas del conflicto',
  'relative_path': '/politica/gobierno-a-rendir-cuentas-sobre-sobre-atencion-a-mujeres-victimas-del-conflicto/'},
 {'title': 'También para Congreso: ya van 129 comités que buscan recolectar firmas',
  'relative_path': '/politica/tambien-para-congreso-ya-van-129-comites-que-buscan-recolectar-firmas/'}]

In [33]:
for n in news:
    # Getting HTML content for news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [34]:
news

[{'title': 'Jennifer Arias influyó en el voto de un representante en la Ley de Garantías',
  'relative_path': '/politica/jennifer-arias-influyo-en-el-voto-de-un-representante-a-la-ley-de-garantias/',
  'datetime': '22 Oct. 2021  - 12:02 p.\xa0m.',
  'author': 'Redacción Política',
  'summary': 'El representante Anatolio Hernández, de la U, preguntó en plena transmisión cómo votar el artículo 125 del Presupuesto de 2022, que modifica la Ley de Garantías. Arias y otra representante le dijeron que votara afirmativamente.',
  'full_text': 'Las redes no perdonan ni olvidan y, aunque ya pasaron tres días desde la aprobación del Presupuesto General de 2022, se está reviviendo un momento de la discusión del proyecto en la Cámara de Representantes. Lea más contexto sobre la discusión alrededor de la Ley de Garantías.  El fragmento del video que circula en Twitter ilustra la votación del artículo que modifica la Ley de Garantías, el tema más polémico del presupuesto, y la postura del representan

In [36]:
# Storing extracted information for further analysis
collection.insert_many(news)